# PLS Regression

🚧 **Page under construction!** 🚧

## Index
- [1. Datos](id-datos)
- [2. Pesos Espaciales](id-pesos-espaciales)
- [3. Normalización](id-normalizacion)
- [4. Detrending](id-detrending)
- [5. PLS Regression en Fortran](id-pls-regression)
- [6. PLS con Scikit-Learn](id-pls-scikit-learn)
- [7. Escritura de Resultados](id-escritura-resultados)



(id-datos)=
## 1. Datos

- **PSL** - Presión a nivel del mar (predictor).
- **TAS** - Temperatura a 2 metros (target).


~~~ 
! FORTRAN
anz_time=84		!ERA5 from 1940 to 2023
l1=11			!start time period for PLS \textbf{\textit{1950}}
l2=80			!end time period for PLS \textbf{\textit{2020}}
!my grid is 384x192, surely it’s the same as Copernicus
MaskTAS=read mask for target 
MaskPSL=[90°W:60°E,20°N:85°N]
~~~

<span style="color:blue">PAPER: MSLP patterns are calculated over the Euro-Atlantic sector [−90, 60]E × [20, 80]N</span>





~~~ 
! FORTRAN
PSL=read seasonal mean pressure data	
!I actually coarsen the grid further for PSL, averaging 2x2 grid cell

TAS=read seasonal mean target data	!but not for the target
~~~

(id-pesos-espaciales)=
## 2. Pesos Espaciales

Pesos espaciales para corregir la diferencia de área entre latitudes.

Se usa luego en la normalización y en la matriz de covarianza.

$$ w(\phi) = \sqrt{\frac{\pi}{180} \cdot \phi} $$

$$ \phi  - latitud $$

~~~ 
! FORTRAN
w(lat) = sqrt(3.14159265 * lat / 180.)
~~~ 

```python
# Python
import numpy as np
w2 = np.cos(np.deg2rad(TAS['latitude']))
w2_broadcasted, TAS_weighted = xr.broadcast(w2, TAS['AnomT2M'])

(id-normalizacion)=
## 3. Normalización
Normalización dividiendo por su varianza espacial

\begin{equation}
    \sigma^2_{\textbf{TAS}} = \frac{\sum_{i,j,t} \textbf{TAS}(i,j,t)^2}{\sum \text{Mask}_{\textbf{TAS}} \cdot (l_2 - l_1 + 1)}
\end{equation}

\begin{equation}
    \textbf{TAS}(i,j,t) = \frac{\textbf{TAS}(i,j,t)}{\sqrt{\sigma^2_{\textbf{TAS}}}}
\end{equation}

\begin{equation}
    \sigma^2_{\textbf{PSL}} = \frac{\sum_{i,j,t} \textbf{PSL}(i,j,t)^2}{\sum \text{Mask}_{\textbf{PSL}} \cdot (l_2 - l_1 + 1)}
\end{equation}

\begin{equation}
    \textbf{PSL}(i,j,t) = \frac{\textbf{PSL}(i,j,t)}{\sqrt{\sigma^2_{\textbf{PSL}}}}
\end{equation}

~~~ 
! FORTRAN
varTAS = sum(TAS(:,:,l1:l2)**2.) / (sum(MaskTAS) * real(l2-l1+1))
TAS(:,:,:) = TAS(:,:,:) / sqrt(varTAS)
varPSL = sum(PSL(:,:,l1:l2)**2.) / (sum(MaskPSL) * real(l2-l1+1))
PSL(:,:,:) = PSL(:,:,:) / sqrt(varPSL)
~~~ 

```python
# Python

# Media y desviación estándar (para estandarizar)
PSL_mean, PSL_std = PSL.mean(dim="year"), PSL.std(dim="year")
TAS_mean, TAS_std = TAS.mean(dim="year"), TAS.std(dim="year")

# Evitar división por cero
PSL_std = PSL_std.where(PSL_std != 0, 1)
TAS_std = TAS_std.where(TAS_std != 0, 1)

# Estandarizar
PSL = (PSL - PSL_mean) / PSL_std
TAS = (TAS - TAS_mean) / TAS_std

(id-detrending)=

## 4. Detrending

<div style="
    background-color: #f8f9fa; 
    padding: 20px; 
    border-radius: 12px; 
    border: 1px solid #ccc; 
    box-shadow: 4px 4px 12px rgba(0,0,0,0.1); 
    text-align: justify; 
    color: #333; 
    font-family: Arial, sans-serif;
    line-height: 1.6;
    max-width: 700px;
    margin: auto;
">
    <h3 style="text-align: center; color: #2c3e50;">📌 Detrending en Análisis de Datos</h3>
    <p>
        El <strong>detrending</strong> es un proceso en el análisis de datos en el que se elimina una tendencia de una serie temporal. 
        Se usa en meteorología para analizar anomalías sin que la tendencia global afecte a los resultados.
    </p>
    <p>
        Matemáticamente, implica ajustar y restar una función f(t)  de la serie de datos x(t):
    </p>
    <pre style="background-color: #e8e8e8; padding: 10px; border-radius: 6px; font-family: monospace;">
X_dtrended = x(t) - f(t)
f(t) = at² + bt + c  (detrending cuadrático)
    </pre>
    <p>
        El propósito de esta operación es garantizar que las variaciones en los datos sean relativas a la 
        variabilidad climática en lugar de tendencias a largo plazo.
    </p>
</div>





Se hace detrending para TAS:

1. Tomar la media ponderada espacialmente sobre Alemania.

2. Ajustar una función cuadrática a la serie temporal.

3. Sustraer esta tendencia de todos los puntos.

\begin{equation}
    \textbf{TAS}_\text{detrended}(i,j,t) = \textbf{TAS}(i,j,t) - \hat{f}(t)
\end{equation}

donde $\hat{f}(t)$ es la tendencia cuadrática ajustada. 

Después, se centra la serie restando la media:

\begin{equation}
    \textbf{TAS}(i,j,t) = \textbf{TAS}(i,j,t) - \overline{\textbf{TAS}}
\end{equation}

Para PSL, simplemente se resta su media: $\textbf{PSL}(i,j,t) = \textbf{PSL}(i,j,t) - \overline{\textbf{PSL}} $


~~~ 
! FORTRAN
TAS = detrend(TAS)
TAS = TAS - mean(TAS)
PSL = PSL - mean(PSL)
~~~ 



```python
#Python
times = TAS['year']
poly_coeffs = Polynomial.fit(times.values, tas_mean_region.values, 2).convert().coef
trend = xr.DataArray(np.polyval(poly_coeffs[::-1], times),dims=["year"], coords={"year": times})
TAS = TAS - trend

```{figure} ../_static/detrending_temperature_anomalies.png
---
width: 75%
name: fig-detrending
---
Anomalías de temperatura después de eliminar la tendencia.



(id-pls-regression)=
## 5. PLS Regression en Fortran
Se implementa la regresión de mínimos cuadrados parciales (PLS).

Se definen matrices

```fortran
! FORTRAN
allocate(U(anz_time,c2),Y(anz_time,c1))		
! make area w-weighted PSL and TAS fields into vectors, put them into U and Y

### 5.1 Matriz de Covarianza

Se define la matriz de covarianza entre predictores  $\mathbf{U}$ y $\mathbf{Y}$ .


\begin{equation}
    \mathbf{COV} = \frac{1}{anz\_time} \mathbf{U}^T \mathbf{Y}
\end{equation}


(id-pls-scikit-learn)=
## 6. PLS con Scikit-Learn

```python
# Python
from sklearn.cross_decomposition import PLSRegression
fac = 4
pls = PLSRegression(n_components=fac)
pls.fit(X_scaled, Y_scaled)
x_weights = pls.x_weights_.T  # (fac, 9966)
y_weights = pls.y_weights_.T  # (fac, 176)

(id-escritura-resultados)=
## 7. Escritura de Resultados
Se guardan los resultados en formato NetCDF para análisis posterior.

```python
xr.DataArray(
    r2, dims=['component','latitude', 'longitude'],
    coords={'longitude': PSL['longitude'], 'latitude': PSL['latitude'], 'component': np.arange(1, fac+1)},
    name='PSLweights').to_netcdf(output_psl)

xr.DataArray(
    q2, dims=['component','latitude', 'longitude'],
    coords={'longitude': TAS['longitude'], 'latitude': TAS['latitude'], 'component': np.arange(1, fac+1)},
    name='TASweights').to_netcdf(output_tas)